In [1]:
from OCR import *
from YoloSplit import *
from owlvit import *
from CLIPstate import *
from state_detection import *
from breaking_state_function import *
from vehicle_detection import *
from chat import *

import shutil
import os

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\Martijn\miniconda3\envs\BEP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# TODO: Breaking state toevoegen?
# TODO: matrix borden detectie/ uitlezen toevoegen
# TODO: weg deel toevoegen --> waar de weg is/ hoe die loopt


def run_program(input):
    # Remove leftover images from previous run of code.
    if os.path.exists("tri-crop"):
        shutil.rmtree("tri-crop")

    dir = os.getcwd()

    for f in os.listdir(dir + "/Crops"):
        os.remove(os.path.join(dir + "/Crops", f))

    # Set name of image file to analyse
    image = input
    print("testing image ", input)

    text_weighted = [
        ["a photo of a person", 0.25],
        ["a photo of a train", 0.4],
        ["a photo of a boat", 0.4],
        ["a photo of a traffic light", 0.45],
        ["a photo of a stop sign", 0.4],
        ["a photo of a cat", 0.4],
        ["a photo of a dog", 0.4],
        ["a photo of a horse", 0.4],
        ["a photo of a sheep", 0.4],
        ["a photo of a cow", 0.4],
        ["a photo of a traffic cone", 0.4],
        ["a photo of a traffic sign", 0.35],
        ["a photo of a ball", 0.4],
        ["a photo of a tractor", 0.4],
        ["a photo of a digital traffic sign", 0.3],
        ["a photo of a digital traffic sign", 0.4],
    ]

    weather_list = [
        "a picture of snowy",
        "a picture of sunny",
        "a picture of rainy",
        "a picture of overcast",
        "a picture of snowy weather",
        "a picture of sunny weather",
        "a picture of rainy weather",
        "a picture of overcast weather",
    ]

    location_list = [
        "a picture of a highway",
        "a picture of a country road",
        "a picture of a motorway",
        "a picture of a city",
        "a picture of a residential area",
    ]

    # classes is a list of all the classes shown above

    classes_orientation = [
        "car_back",
        "car_side",
        "car_front",
        "bus_back",
        "bus_side",
        "bus_front",
        "truck_back",
        "truck_side",
        "truck_front",
        "motorcycle_back",
        "motorcycle_side",
        "motorcycle_front",
        "bicycle_back",
        "bicycle_side",
        "bicycle_front",
    ]

    classes_owl = [x[0][13:] for x in text_weighted]

    # splits image into 3 parts, outside-view, rear-view, and speed
    # saves to tri-crop/predict/crops/outside-view
    # saves to tri-crop/predict/crops/rear-view
    # saves to tri-crop/predict/crops/speed
    tri_crop_results = yolo_tri_crop("images/" + image)

    # detects number with OCR in file, specified by its path
    car_speed = easyocr_detect(
        os.path.join(dir, "tri-crop/predict/crops/speed/" + image)
    )

    # does a zero shot object detection on an image and returns boxes, labels, and scores
    owl_boxes, owl_labels, owl_scores = owlvit_object_detect(
        text_weighted,
        os.path.join(dir, "tri-crop/predict/crops/outside-view/" + image),
    )

    weather, location = CLIP_state_detect(
        os.path.join(dir, "tri-crop/predict/crops/outside-view/" + image),
        weather_list,
        location_list,
    )

    # detecteerd de voertuigen
    # detecteerd de voertuigen
    image_front = "tri-crop/predict/crops/outside-view/" + image

    vehicles_detected = vehicle_detection(image_front)

    # maakt het dataframe
    df = dataframe_bouwen(
        owl_labels,
        owl_boxes,
        owl_scores,
        classes_owl,
        vehicles_detected,
        classes_orientation,
        tri_crop_results,
        image,
    )

    # Elke crop maken uit de tabel en foto naam aan tabel toevoegen
    fotonaam = []
    for row in range(df.shape[0]):
        fotonaam = crop_and_save_image(row, df, image_front, fotonaam)
    df["foto_naam"] = fotonaam

    # bepaald de state een verkeersbord of verkeerslicht

    for row in range(df.shape[0]):
        if str(df.iloc[row]["class_naam"]) == "traffic sign":
            Traffic_sign(row, df)

        elif str(df.iloc[row]["class_naam"]) == "traffic light":
            Traffic_light(row, df)

        elif (
            str(df.iloc[row]["state"]) == "back"
            and str(df.iloc[row]["class_naam"]) == "car"
        ):
            Braking(row, df)

    df = position(df, image)

    prompt, response = ChatGPT(df, car_speed, location, weather)

    print(prompt)
    print(response)

    text_file = open("Output.txt", "w")
    text_file.write(prompt)
    text_file.write("")
    text_file.write(response)
    text_file.close()

    # reset the variables
    df = None
    car_speed = None
    tri_crop_results = None


# df.to_csv("C:/Users/Mees/Desktop/dataframe_voor_depth.csv")

# for image in os.listdir("images"):
#     input("PRESS ENTER TO CONTINUE")
#     run_program(image)




In [27]:
image = "vraag 31.jpg"
run_program(image)

testing image  vraag 31.jpg


image 1/1 C:\Users\Martijn\Desktop\coding-projects\BEP\GPT-DT\images\Vraag 31.jpg: 480x640 1 outside-view, 1 rear-view, 1 speed, 243.4ms
Speed: 3.0ms preprocess, 243.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to tri-crop\predict
YOLOv5  2023-5-10 Python-3.9.16 torch-2.0.0 CPU



'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


Fusing layers... 
Model summary: 476 layers, 76226352 parameters, 0 gradients, 110.1 GFLOPs
Adding AutoShape... 


1/1 [==============================] - 0s 138ms/step
Pedestrian crossing
Assume you are driving in The Netherlands. You are driving in a country road at 35 km/h. The weather condition is overcast weather. This is your front view; You see the following cars: A car approaching from adjacently straight infront. You see the following traffic signs: A "Pedestrian crossing" traffic sign. You see the following traffic lights: There are no traffic lights in sight. You see the following pedestrians: There are no pedestrians in sight. You see the following bicyclist: There are no bicycles in sight. Additionally, you see: there are no more objects than the ones mentioned above. This is your rear view: You see the following  cars: There are no significant objects behind you.Given the described situation above, what would you do: 'Let go of the gas pedal', 'Brake' or 'Do nothing'. Show the three options I gave you and pick your answer. Give your thorough reason behind it.


Answer: Brake.

The reas